# Atmospheric Model Intercomparison Project (AMIP) Validation

To investigate global stilling, we want to force the models realistic historical SST forcing in order to force the models with the observed realization. To do this we use the `amip` model runs.

__Variables analyzed__
<!-- - `uas`: eastward wind component (usually 10 m) [$m \ s^{-1}$]
- `vas`: northward wind component (usually 10 m) [$m \ s^{-1}$] -->
- `sfcWind`: Near-Surface Wind Speed [$m \ s^{-1}$]
- _sfcWind anomaly_: artificially constructed according to $x_i-\bar{x}_{\text{time}}$

Models are available on Andromeda (The BC Cluster) at `/data/projects/bccg/CMIP6/amip/mon/sfcWind`. Models are at _monthly_ resolution and aggregated _yearly_ before any trend analysis is analyzed.

__Steps to connect to BC Cluster__
1. Install Remote SSH and Remote X11 extensions in VScode
2. `ssh -Y username@andromeda.bc.edu`
3. Enter password
4. You are now in your home directory located at `~/mmfs1/data/_username_`
5. `module load python/3.9.0` $\leftarrow$ add to .tcshrc file to automatically load every login

__To start an interactive session__:

`interactive -t [DD-hh:mm] [-N nodes) [-n tasks] [-c cpus-per-task] [-m gb] [-p partition] [-G #] [-X] [-h]`

Options:  
- `t`: Wall Time (default is 4 hours)
- `N`: Number of nodes (default is 1) 
- `m`: GB of Memory per node (default is 4GB) 
- `n`: Number of tasks per node (default is 1) 
- `c`: Number of cpu cores per task (default is 4) 
- `X`: Use X11 
- `p` <partition name>: Use the partition specified (default is shared) 
- `G` #: Specify the number of GPUs per gpu node
- `h`: help

My default command: `interactive`

Useful command to monitor usage: `htop`

__Note__: This will not allow you to run a Jupyter Notebook interactively, only scripts. To run notebooks interactively use `dask_jobqueue.SLURMCluster()`

__Getting Conda up and Running__
1. `module load anaconda/2023.07-p3.11`
2. `conda init tcsh`
3. `conda create -n _envname_ python=3.11`
4. `conda activate _envname_`

This will create a conda environment in the `/mmfs1/data/_username_/.conda/envs/_envname_` directory. 

To automatically use this environment on login use add `conda activate _envname_` to your `.tcshrc` file.

__For a faster environment solver__
1. `conda install -n _envname_ conda-libmamba-solver`
2. `conda config --set solver libmamba`

<!-- __Export Environment__: `conda env export --no_builds > environment.yml` -->

__Note__: To use `matplotlib` we must install $\LaTeX$. Jupyter Notebooks use MathJax under the hood which is why we only need to install if it using $\LaTeX$ in python.


In [56]:
import matplotlib.pyplot as plt
from matplotlib import colors
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LatitudeFormatter, LongitudeFormatter
import nc_time_axis
import numpy as np
import pandas as pd
import pwlf
import xarray as xr
import cf_xarray as cfxr   # to use cf_xarray attributes
import regionmask
from glob import glob
import scienceplots
%matplotlib inline
plt.style.use(["nature", "notebook"])

xr.set_options(keep_attrs=True)
%load_ext rich
from rich import print  # pretty printing
from tqdm import tqdm  # progress bar
import warnings  # deal with warnings

# Playing nice with CMIP6
# from xmip.preprocessing import combined_preprocessing
# from xclim.ensembles import create_ensemble, ensemble_mean_std_max_min

The rich extension is already loaded. To reload it, use:
  %reload_ext rich


## Spin up dask cluster

In [57]:
# For using dask cluster
# from dask_jobqueue import SLURMCluster
# from distributed import Client

# Change params as needed
# num_jobs = 3
# walltime = '02:00:00'

# cluster = SLURMCluster(
#     memory="8g",
#     processes=1,
#     cores=4, 
#     log_directory='/mmfs1/data/valencig/worker-logs/',
#     walltime=walltime,
# )

# # Assign the cluster to our Client
# client = Client(cluster)

# # Block progress until workers have spawned
# client.wait_for_workers(num_jobs)

# client

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:5                                                                                    │
│                                                                                                  │
│    2 num_jobs = 3                                                                                │
│    3 walltime = '02:00:00'                                                                       │
│    4                                                                                             │
│ ❱  5 cluster = SLURMCluster(                                                                     │
│    6 │   memory="8g",                                                                            │
│    7 │   processes=1,                                                                            │
│    8 │   cores=4,                                                                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'SLURMCluster' is not defined

## Helper Functions

In [ ]:
def mask_data(data, map, regions: None, drop=False, reverse=False):
    """Mask xarray data based on region names

    Args:
        data (xarray dataset): xarray dataset to mask
        map (regionmask): regionmask object
        regions (list or None): list of region names to mask. if None, all regions are taken. Defaults to None.
        drop (bool, optional): Whether to drop when masking. Defaults to False.
        reverse (bool, optional): Whether to mask the inverse of the regions. Defaults to False.

    Returns:
        xarray dataset: masked dataset
    """
    # Load the region mask
    mask = map.mask(data.cf['X'], data.cf['Y'])
    # Extract keys for the region
    id_dict = map.region_ids
    # Get region names
    if regions is None:
        names = id_dict.keys()
    else:
        # Coerce region names to uppercase
        regions = [region.upper() for region in regions]
        names = [name for name in id_dict.keys() if str(name).upper() in regions]
        assert len(names) == len(regions), 'Not enough regions found'
    # Get the key for the regions
    keys = [id_dict[name] for name in names]
    # Apply the mask to the data
    if reverse:
        masked_data = data.where(~mask.isin(keys), drop=drop)
    else:
        masked_data = data.where(mask.isin(keys), drop=drop)
    return masked_data

## Create the ensemble

### Model names

In [ ]:
# Get all model names
model_folders = glob('/Volumes/valencig@bc/data/amip/mon/sfcWind/*')
model_names = [f.split('/')[-1] for f in model_folders]
model_names = sorted(model_names)

### Remove certain models


Removal keys: 
- `__ km`: Nominal model resolution. We are looking for models of $\leq$ 1 degrees (100 km or less).

Data about models manually searched from [here](https://wcrp-cmip.github.io/CMIP6_CVs/docs/CMIP6_source_id.html).

In [ ]:
bad_models = [
    'ACCESS-CM2',       # 250 km
    'ACCESS-ESM1-5',    # 250 km
    'BCC-ESM1',         # 250 km
    'CESM2-FV2',        # 250 km
    'CESM2-WACCM-FV2',  # 250 km
    'CNRM-CM6-1',       # 250 km
    'CNRM-ESM2-1',      # 250 km
    'CanESM5',          # 500 km
    'EC-Earth3-Veg-LR', # 250 km
    'FGOALS-g3',        # 250 km
    'GISS-E2-1-G',      # 250 km
    'GISS-E2-2-G',      # 250 km
    'HadGEM3-GC31-LL',  # 250 km
    'ICON-ESM-LR',      # 250 km
    'IITM-ESM',         # 250 km
    'IPSL-CM6A-LR',     # 250 km
    'KACE-1-0-G',       # 250 km
    'KIOST-ESM',        # 250 km
    'MIROC-ES2L',       # 500 km
    'MIROC6',           # 250 km
    'MPI-ESM-1-2-HAM',  # 250 km
    'MPI-ESM1-2-LR',    # 250 km
    'NESM3',            # 250 km
    'NorESM2-LM',       # 250 km
    'UKESM1-0-LL'       # 250 km
]

model_names = [model for model in model_names if model not in bad_models]
n_models = len(model_names)
print(f'{n_models} models found...')
print('Model names:')
model_names

### Load datasets

In [ ]:
# Create list to hold datasets
ensemble = dict()
# Create dictionary to display indices of models
realizations_dict = dict()
with warnings.catch_warnings():  # suppress warnings from xarray bookeeping
    warnings.simplefilter("ignore")
    for model in tqdm(model_names, desc='Loading models'):
        # Sort to make sure realization indices increasing
        paths = sorted(glob(f'/Volumes/valencig@bc/data/amip/mon/sfcWind/{model}/*'))
        # Extract first realization
        realization = paths[0].split('_')[-3]
        # Add to dictionary
        realizations_dict[model] = realization
        # Filter paths to only include selected realization
        realization_paths = [p for p in paths if realization in p]
        
        # Create ensembles
        # ens = create_ensemble(paths, realizations=realizations)
        # Combine realizations
        # keep_vars = ['sfcWind_mean', 'sfcWind_stdev', 'sfcWind_max', 'sfcWind_min']
        # reduced = ensemble_mean_std_max_min(ens)[keep_vars]
        
        # Load data
        ds = xr.open_mfdataset(realization_paths, combine='by_coords', use_cftime=True)
        # Convert the calendar to standard to merge across models
        calendar_corrected = ds.convert_calendar('standard', align_on='year', use_cftime=True)
        # Check times
        sliced = calendar_corrected.sel(time=slice('1979', '2014'))
        # Add to ensemble
        ensemble[model] = sliced
        
    print(realizations_dict)

## Imports GSOD stations from [Zeng et. al (2019).](https://www.nature.com/articles/s41558-019-0622-6)

In [ ]:
obs_path = 'data/Zeng-2019/41558_2019_622_MOESM2_ESM.xlsx'
# GSOD data from Zeng (2019) -> not using HADISD
excel = pd.ExcelFile(obs_path)
# See sheet names
stations = excel.parse('stations')
obs_lats = stations['lats']
obs_lons = stations['lons']  # don't want negative degrees
station_nums = stations['stations']
obs_winds = excel.parse('winds')
# Construct years out of numpy
years = np.arange(1979, 2015)

gsod = xr.Dataset(
            data_vars={
                'GSOD': (["station", "year"], obs_winds.to_numpy()[:, 2:-3]) 
                # First col is index, second col is 1978, last col is 2017
            },
            coords={
                'station': station_nums,
                # 'year': pd.date_range(start='1978', end='2015', freq='Y'), # [1978, 2014]
                'year': years,
                'lon': ("station", obs_lons),
                'lat': ("station", obs_lats)
            }
    )
# Get yearly data
anom = gsod.GSOD - gsod.GSOD.mean('year')
obs_anom = anom.groupby('year').mean('station') # means

# Compute gsod piecewise linear fit
gsod_pwlf = pwlf.PiecewiseLinFit(years, obs_anom)
breaks = gsod_pwlf.fit(2)
break_year = int(breaks[1])
slopes = gsod_pwlf.calc_slopes()*10
p_vals = gsod_pwlf.p_values(method='non-linear')[1:]  # betas are [intercept, slope1, slope2, ...]
# print(f'Piecewise Parameters\n\tBreakpoint: {break_year}\n\tDecadal Slopes: {slopes}\n\tP-Values: {p_vals}')
y_hat = gsod_pwlf.predict(years)

# Make plot
fig, ax = plt.subplots(figsize=(7, 5), nrows=1, ncols=1)
ax_min = min(obs_anom) - 0.05
ax_max = max(obs_anom) + 0.05
ax.plot(years, obs_anom, linestyle='--', color='blue')
ax.plot(years, y_hat, '-', color='k')
ax.vlines(break_year, ax_min, ax_max, color='gray', linestyle='--')
ax.set_ylim(ax_min, ax_max)
ax.set_xlim(1978, 2014)
# ax.fill_between(years, obs_q.isel(quantile=0), obs_q.isel(quantile=1), color='red', alpha=0.1)
# Add small annotation
ins = ax.inset_axes([0.7, 0.6, 0.25, 0.35])
pos = slopes > 0
neg = slopes < 0
l = np.array([1, 2])
for i, (m, p) in enumerate(zip(slopes, p_vals)):
    i+=1
    if m > 0:
        rects = ins.bar(i, m, color='red')
    else:
        rects = ins.bar(i, m, color='blue')
    if p < 0.01:
        ins.bar_label(rects, ['p$<$0.01'])
# Partition the percentile values to be able to draw large numbers in
# white within the bar, and small numbers in black outside the bar.
ins.hlines(0, 0.25, 2.75, color='k')
# Remove spines
ins.spines['top'].set_visible(False)
ins.spines['right'].set_visible(False)
ins.spines['bottom'].set_visible(False)
# Remove ticks
ins.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False,
)
# Set ticks
ymin = int(min(slopes)/0.1)*0.1-0.1
ymax = int(max(slopes)/0.1)*0.1+0.1
ins.set_yticks(np.arange(ymin, ymax, 0.1))
ins.tick_params(axis='y', labelsize=12)
ins.set_ylim(ymin, ymax)
ins.set_ylabel('Trend\n[m s$^{-1}$ decade$^{-1}$]', fontsize=12)

# Labels
ax.set_xlabel('Year')
ax.set_ylabel('NSWS Anomaly [m s$^{-1}$]')
ax.set_title('GSOD stations from Zeng et. al (2019)')
plt.show()

## Plots Average Land Trend

Piecewise linear trends are calculated using `fit` from the `pwlf` package. P-values are determined using the `p_values()` function with `method='non-linear'`. From the docs: 

"The non-linear regression problem is when you don’t know the breakpoint locations (e.g. when using the fit, fitfast, and fit_guess functions)."



In [ ]:
# Masking regions
land_region = regionmask.defined_regions.natural_earth_v5_0_0.land_110  # Land has value 0
countries = regionmask.defined_regions.natural_earth_v5_0_0.countries_110

fig = plt.figure(figsize=(14, int(n_models*3.5)), constrained_layout=True)
gs = fig.add_gridspec(n_models, 2, width_ratios=[1.5, 1])

# Save data to dataframe
piecewise_params = pd.DataFrame(
    dict(
        source='GSOD',
        break_year=break_year,
        slope_1=slopes[0],
        slope_2=slopes[1],
        p_val_1=p_vals[0],
        p_val_2=p_vals[1],
        sig_1=p_vals[0] < 0.01,
        sig_2=p_vals[1] < 0.01,
    ), index=[0]
)

for i, model in enumerate(tqdm(ensemble.keys(), desc='Plotting models')):
    ds = ensemble[model]
    # Get anomaly
    ds = ds - ds.cf.mean('T')
    # mask to land
    land = mask_data(ds, land_region, ['land'], drop=True)
    # Eliminate the planet of Hoth
    land = mask_data(land, countries, ['greenland'], reverse=True, drop=True)
    land = land.where((land['lat']>-59).compute(), drop=True)  # antarctica
    land = land.where((land['lat']<70).compute(), drop=True)  # Northern canada
    # land = land.where(land,drop=True)
    # Create axes for plotting
    map = fig.add_subplot(gs[i, 0], projection=ccrs.PlateCarree())
    ts = fig.add_subplot(gs[i, 1])
    # Plot map
    trend = (
        land['sfcWind'].cf.groupby('T.year').mean()
        .polyfit('year', deg=1, skipna=True)
        .polyfit_coefficients.sel(degree=1)*10  # decadal
    )
    im = trend.plot(ax=map, vmin=-0.2, vmax=0.2, cmap='coolwarm', transform=ccrs.PlateCarree(), add_colorbar=False)
    cb = plt.colorbar(im, orientation="vertical", pad=0.01, extend='both', shrink=0.7)
    cb.set_label(label='Decadal Trend [m/s]')
    # Plot time series
    land_nsws = land['sfcWind'].cf.groupby('T.year').mean().cf.mean(['X', 'Y']).values
    ts.plot(years, land_nsws, label=model, color='blue', linestyle='--')
    ts.plot(years, obs_anom, label='GSOD', color='k', linestyle='--')
    ts.plot(years, gsod_pwlf.predict(years), color='k')
    # Add model linear fit
    model_pwlf = pwlf.PiecewiseLinFit(years, land_nsws)
    breaks = model_pwlf.fit(2)
    break_year = int(breaks[1])
    slopes = model_pwlf.calc_slopes()*10  # make decadal
    p_vals = model_pwlf.p_values(method='non-linear')[1:]  # betas are [intercept, slope1, slope2, ...]
    # print(f'Piecewise Parameters\n\tBreakpoint: {break_year}\n\tDecadal Slopes: {slopes}\n\tP-Values: {p_vals}')
    ts.plot(years, model_pwlf.predict(years), color='blue')
    ax_min = min(min(obs_anom), min(land_nsws)) - 0.05
    ax_max = max(max(obs_anom), max(land_nsws)) + 0.05
    ts.vlines(break_year, ax_min, ax_max, color='gray', linestyle='--')
    ts.set_ylim(ax_min, ax_max)
    # Annotation plot for trend
    ins = ts.inset_axes([0.7, 0.6, 0.25, 0.35])
    pos = slopes > 0
    neg = slopes < 0
    l = np.array([1, 2])
    for i, (m, p) in enumerate(zip(slopes, p_vals)):
        i+=1
        if m > 0:
            rects = ins.bar(i, m, color='red')
        else:
            rects = ins.bar(i, m, color='blue')
        if p < 0.05:
            ins.bar_label(rects, ['p$<$0.05'])
    # Partition the percentile values to be able to draw large numbers in
    # white within the bar, and small numbers in black outside the bar.
    ins.hlines(0, 0.25, 2.75, color='k')
    # Remove spines
    ins.spines['top'].set_visible(False)
    ins.spines['right'].set_visible(False)
    ins.spines['bottom'].set_visible(False)
    # Remove ticks
    ins.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom=False,      # ticks along the bottom edge are off
        top=False,         # ticks along the top edge are off
        labelbottom=False,
    )
    # Set ticks
    ymin = int(min(slopes)/0.1)*0.1-0.1
    ymax = int(max(slopes)/0.1)*0.1+0.1
    ins.set_yticks(np.arange(ymin, ymax, 0.1))
    ins.tick_params(axis='y', labelsize=10)
    ins.set_ylim(ymin, ymax)
    ins.set_ylabel('Trend\n[m s$^{-1}$ decade$^{-1}$]', fontsize=10)
    
    
    # Map options
    map.add_feature(cfeature.COASTLINE)
    # map.add_feature(cfeature.BORDERS)
    map.set_title(f'{model} Decadal NSWS Trend')
    gl = map.gridlines(
        crs=ccrs.PlateCarree(), draw_labels=True, which='both',
        linewidth=1, color='k', alpha=1, linestyle='--'
    )
    gl.right_labels = None
    gl.top_labels = None
    gl.xlines = None
    gl.ylines = None
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    # Increase the ticksize
    gl.xlabel_style = {'size': 12, 'color': 'k', 'rotation':30, 'ha':'right'}
    gl.ylabel_style = {'size': 12, 'color': 'k', 'weight': 'normal'}

    # Time series options
    ts.set_title(f'{model} Land NSWS')
    ts.set_ylabel('NSWS Anomaly [m s$^{-1}$]')
    # Save results of piecewise fit
    piecewise_params = pd.concat([
        piecewise_params, 
        pd.DataFrame(
            dict(
                source=model,
                break_year=break_year,
                slope_1=slopes[0],
                slope_2=slopes[1],
                p_val_1=p_vals[0],
                p_val_2=p_vals[1],
                sig_1=p_vals[0] < 0.05,
                sig_2=p_vals[1] < 0.05
                ), index=[0]
        )
        ], ignore_index=True)

plt.savefig('figures/amip_land_trends.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
print('Parameters of piecewise regression:')
print(piecewise_params)
plt.show()

## Heatmap of prediction

Using Giorgi scientific regions (from Giorgi and Franciso, 2000). From regionmask:

"The Giorgi reference regions, rectangular regions proposed in Giorgi and Francisco, 2000 were used in the third and fourth assessment reports of the Intergovernmental Panel on Climate Change (IPCC)."

For observation data, all regions with <40 stations are ignored. This is to ensure reliable consistency


In [ ]:
# Masking regions
giorgi = regionmask.defined_regions.giorgi
land_region = regionmask.defined_regions.natural_earth_v5_0_0.land_110  # Land has value 0
regions = giorgi.abbrevs
regions.remove('GRL')  # don't do greenland

# Get trends for observational data
regressed = dict(source='Observation')
bad_regions = []
# Add global trends to dataframe
glob_trend = obs_anom.polyfit('year', deg=1, skipna=True).polyfit_coefficients.sel(degree=1)*10
regressed['Global'] = glob_trend.values
obs_mask = giorgi.mask(gsod.lon, gsod.lat)
for r in regions:
    # Extract keys for the region
    key = giorgi.map_keys(r)
    obs = gsod.where(obs_mask == key, drop=True)
    obs = obs - obs.cf.mean('year')
    if obs.station.size < 40:
        bad_regions.append(r)
    else:
        decade_trend = obs.GSOD.mean('station').polyfit('year', deg=1, skipna=True).polyfit_coefficients.sel(degree=1)*10
        regressed[r] = decade_trend.values
# Create dataframe to store regressions
regions = [r for r in regions if r not in bad_regions]
region_trends = pd.DataFrame(
    columns=['source', *regions]
)
# Add GSOD data
region_trends = pd.concat(
    [
        region_trends,
        pd.DataFrame(regressed, index=[0])
    ], ignore_index=True
)
    
for i, model in enumerate(tqdm(ensemble.keys(), desc='Deriving Regional Trends')):
    ds = ensemble[model]
    # Get anomaly
    ds = ds - ds.cf.mean('T')
    # Resample to yearly
    da = ds['sfcWind'].cf.groupby('T.year').mean()
    # mask to land
    land = mask_data(da, land_region, ['land'], drop=True)
    # Eliminate the planet of Hoth
    land = land.where((land['lat']>-59).compute(), drop=True)  # antarctica
    land = land.where((land['lat']<70).compute(), drop=True)  # Northern canada
    # For each region determine the average trend
    regressed = dict(source=model)
    # Add Global trend
    glob_trend = land.cf.mean(['X', 'Y']).polyfit('year', deg=1, skipna=True).polyfit_coefficients.sel(degree=1)*10
    regressed['Global'] = glob_trend.values
    for r in regions:
        masked = mask_data(land, giorgi, [r], drop=True)
        decade_trend = masked.cf.mean(['X', 'Y']).polyfit('year', deg=1, skipna=True).polyfit_coefficients.sel(degree=1)*10
        regressed[r] = decade_trend.values    
    # Add to dataframe
    region_trends = pd.concat(
        [
            region_trends,
            pd.DataFrame(regressed, index=[0])
        ], ignore_index=True
    )
region_trends

### Heatmap of prediction

In [ ]:
x = 12
y = 12
columns = [*regions, 'Global']
fig, axes = plt.subplots(figsize=(x, y), nrows=2)
fig.subplots_adjust(hspace=.4)
data = region_trends[columns].T
# Plot 1: trend
ax = axes.flat[0]
im = ax.pcolor(data, cmap='bwr', vmin=-0.15, vmax=0.15)
# Add colorbar
cb = plt.colorbar(im, orientation='vertical', pad=0.01, extend='both')
cb.set_label(label='Decadal Trend [m s$^{-1}$ decade$^{-1}$]', fontsize=14)

# Style options
ax.set_ylabel('Giorgi Region', fontsize=14)
ax.set_title('NSWS Wind Trends', fontsize=16)

# Plot 2: Aggreement with observation
ax = axes.flat[1]
signs = np.copysign(np.ones_like(data), (data)).to_numpy()
obs_sign = signs[:, 0]
obs_sign = signs[:, 0].reshape(-1, 1)
agreements = np.where(signs == obs_sign, 1, 0)
im = ax.pcolor(agreements, cmap=colors.ListedColormap(['white', 'Green']))
cb = plt.colorbar(im, orientation='vertical', pad=0.01, ticks=[0.25, 0.75])
cb.set_ticklabels(['Disagree', 'Agree'], rotation=60)
cb.set_label(label='Sign agreement with observation', fontsize=14)

# Options for both plots
for ax in axes.flat:
    # set side axis
    ax.set_ylim(ax.get_ylim())
    ax.set_yticks(np.arange(len(columns))+0.5)
    ax.set_yticklabels(columns, fontsize=10)
    
    # set top axis
    ax.set_xlim(ax.get_xlim())
    ax.set_xticks(np.arange(len(region_trends['source']))+0.5)
    ax.set_xticklabels(region_trends['source'], rotation=90, fontsize=10)
    
    # For grid lines
    ax.set_yticks(np.arange(0, len(columns)), minor=True)
    ax.set_xticks(np.arange(0, len(region_trends['source'])), minor=True)
    ax.grid(True, axis='both', which='minor', color='k', linewidth=2)
    ax.tick_params(axis='both', which='major', length=1)
    
    # Highlighted models
    for i in [4, 5]:  
        label = ax.xaxis.get_ticklabels()[i]
        label.set_bbox(dict(facecolor='none', edgecolor='red'))

plt.savefig('figures/amip_regional_trends.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
plt.show()

## Investigate best models

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

ax.plot(years, obs_anom, label='GSOD', color='k', linestyle='--')
ax.plot(years, gsod_pwlf.predict(years), color='k')
for model, color in zip(['CESM2', 'CESM2-WACCM'], ['blue', 'red']):
    ds = ensemble[model]
    # Get anomaly
    ds = ds - ds.cf.mean('T')
    # Resample to yearly
    da = ds['sfcWind'].cf.groupby('T.year').mean()
    # mask to land
    land = mask_data(da, land_region, ['land'], drop=True)
    # Eliminate the planet of Hoth
    land = land.where((land['lat']>-59).compute(), drop=True)  # antarctica
    land = land.where((land['lat']<70).compute(), drop=True)  # Northern canada
    # Plot time series
    land_nsws = land.cf.mean(['X', 'Y']).values
    ax.plot(years, land_nsws, label=model, linestyle='--', color=color)
    # Add model linear fit
    model_pwlf = pwlf.PiecewiseLinFit(years, land_nsws)
    breaks = model_pwlf.fit(2)
    break_year = int(breaks[1])
    slopes = model_pwlf.calc_slopes()*10  # make decadal
    p_vals = model_pwlf.p_values(method='non-linear')[1:]  # betas are [intercept, slope1, slope2, ...]
    # print(f'Piecewise Parameters\n\tBreakpoint: {break_year}\n\tDecadal Slopes: {slopes}\n\tP-Values: {p_vals}')
    ax.plot(years, model_pwlf.predict(years), color=color)
    ax_min = min(min(obs_anom), min(land_nsws)) - 0.05
    ax_max = max(max(obs_anom), max(land_nsws)) + 0.05
    # ax.vlines(break_year, ax_min, ax_max, color='gray', linestyle='--')
    ax.set_ylim(ax_min, ax_max)
    
ax.legend()
ax.set_xlabel('Year')
ax.set_ylabel('NSWS Anomaly [m s$^{-1}$]')
ax.set_title('Comparison of Best Models')

plt.savefig('figures/amip_best_models.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
plt.show()

## Ocean Trend

In [ ]:
# Masking regions
land_region = regionmask.defined_regions.natural_earth_v5_0_0.land_110  # Land has value 0
countries = regionmask.defined_regions.natural_earth_v5_0_0.countries_110

fig = plt.figure(figsize=(14, int(n_models*3.5)), constrained_layout=True)
gs = fig.add_gridspec(n_models, 2, width_ratios=[1.5, 1])

ocean_piecewise_params = pd.DataFrame(
    columns = [
        'source',
        'break_year',
        'slope_1',
        'slope_2',
        'p_val_1',
        'p_val_2',
        'sig_1',
        'sig_2'
    ]
)

for i, model in enumerate(tqdm(ensemble.keys(), desc='Plotting models')):
    ds = ensemble[model]
    # Get anomaly
    ds = ds - ds.cf.mean('T')
    # mask to land
    ocean = mask_data(ds, land_region, ['land'], drop=True, reverse=True)
    # Create axes for plotting
    map = fig.add_subplot(gs[i, 0], projection=ccrs.PlateCarree())
    ts = fig.add_subplot(gs[i, 1])
    # Plot map
    trend = (
        ocean['sfcWind'].cf.groupby('T.year').mean()
        .polyfit('year', deg=1, skipna=True)
        .polyfit_coefficients.sel(degree=1)*10  # decadal
    )
    im = trend.plot(ax=map, vmin=-0.2, vmax=0.2, cmap='coolwarm', transform=ccrs.PlateCarree(), add_colorbar=False)
    cb = plt.colorbar(im, orientation="vertical", pad=0.01, extend='both', shrink=0.7)
    cb.set_label(label='Decadal Trend [m/s]')
    # Plot time series
    ocean_nsws = ocean['sfcWind'].cf.groupby('T.year').mean().cf.mean(['X', 'Y']).values
    ts.plot(years, ocean_nsws, label=model, color='blue', linestyle='--')
    # Add model linear fit
    model_pwlf = pwlf.PiecewiseLinFit(years, ocean_nsws)
    breaks = model_pwlf.fit(2)
    break_year = int(breaks[1])
    slopes = model_pwlf.calc_slopes()*10  # make decadal
    p_vals = model_pwlf.p_values(method='non-linear')[1:]  # betas are [intercept, slope1, slope2, ...]
    # print(f'Piecewise Parameters\n\tBreakpoint: {break_year}\n\tDecadal Slopes: {slopes}\n\tP-Values: {p_vals}')
    ts.plot(years, model_pwlf.predict(years), color='blue')
    ax_min = min(min(obs_anom), min(ocean_nsws)) - 0.05
    ax_max = max(max(obs_anom), max(ocean_nsws)) + 0.05
    ts.vlines(break_year, ax_min, ax_max, color='gray', linestyle='--')
    ts.set_ylim(ax_min, ax_max)
    # Annotation plot for trend
    ins = ts.inset_axes([0.7, 0.6, 0.25, 0.35])
    pos = slopes > 0
    neg = slopes < 0
    l = np.array([1, 2])
    for i, (m, p) in enumerate(zip(slopes, p_vals)):
        i+=1
        if m > 0:
            rects = ins.bar(i, m, color='red')
        else:
            rects = ins.bar(i, m, color='blue')
        if p < 0.05:
            ins.bar_label(rects, ['p$<$0.05'])
    # Partition the percentile values to be able to draw large numbers in
    # white within the bar, and small numbers in black outside the bar.
    ins.hlines(0, 0.25, 2.75, color='k')
    # Remove spines
    ins.spines['top'].set_visible(False)
    ins.spines['right'].set_visible(False)
    ins.spines['bottom'].set_visible(False)
    # Remove ticks
    ins.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom=False,      # ticks along the bottom edge are off
        top=False,         # ticks along the top edge are off
        labelbottom=False,
    )
    # Set ticks
    ymin = int(min(slopes)/0.1)*0.1-0.1
    ymax = int(max(slopes)/0.1)*0.1+0.1
    ins.set_yticks(np.arange(ymin, ymax, 0.1))
    ins.tick_params(axis='y', labelsize=10)
    ins.set_ylim(ymin, ymax)
    ins.set_ylabel('Trend\n[m s$^{-1}$ decade$^{-1}$]', fontsize=10)
    
    
    # Map options
    map.add_feature(cfeature.COASTLINE)
    # map.add_feature(cfeature.BORDERS)
    map.set_title(f'{model} Decadal NSWS Trend')
    gl = map.gridlines(
        crs=ccrs.PlateCarree(), draw_labels=True, which='both',
        linewidth=1, color='k', alpha=1, linestyle='--'
    )
    gl.right_labels = None
    gl.top_labels = None
    gl.xlines = None
    gl.ylines = None
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    # Increase the ticksize
    gl.xlabel_style = {'size': 12, 'color': 'k', 'rotation':30, 'ha':'right'}
    gl.ylabel_style = {'size': 12, 'color': 'k', 'weight': 'normal'}

    # Time series options
    ts.set_title(f'{model} Ocean NSWS')
    ts.set_ylabel('NSWS Anomaly [m s$^{-1}$]')
    # Save results of piecewise fit
    ocean_piecewise_params = pd.concat([
        ocean_piecewise_params, 
        pd.DataFrame(
            dict(
                source=model,
                break_year=break_year,
                slope_1=slopes[0],
                slope_2=slopes[1],
                p_val_1=p_vals[0],
                p_val_2=p_vals[1],
                sig_1=p_vals[0] < 0.05,
                sig_2=p_vals[1] < 0.05
                ), index=[0]
        )
        ], ignore_index=True)

plt.savefig('figures/amip_ocean_trends.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
# print('Parameters of piecewise regression:')
print(ocean_piecewise_params)
plt.show()

## Kill Dask cluster

In [ ]:
client.shutdown()